In [3]:
!pip install selenium

     |████████████████████████████████| 979 kB 4.8 MB/s eta 0:00:01
  Using cached trio-0.20.0-py3-none-any.whl (359 kB)
  Using cached trio_websocket-0.9.2-py3-none-any.whl (16 kB)
  Using cached outcome-1.1.0-py2.py3-none-any.whl (9.7 kB)
  Using cached wsproto-1.1.0-py3-none-any.whl (24 kB)
  Using cached h11-0.13.0-py3-none-any.whl (58 kB)


In [4]:
#import the libraries as needed
from selenium import webdriver
from selenium import webdriver   # for webdriver
from selenium.webdriver.support.ui import WebDriverWait  # for implicit and explict waits
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import numpy as np
import json
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm # to create loadbard in for loop
import os.path

In [5]:
# need to precise the option for date, nbr of metada for call transcript in the url paramater 
# https://en.wikipedia.org/wiki/Query_string
nbr_earning_call_metada = 50 # I think 50 is max but can easyily be adapted, with for loop or something like that
date_of_request= "2022-04-28"
url = "https://seekingalpha.com/api/v3/articles"+"?cacheBuster="\
      +str(date_of_request)\
      +"&filter[category]=earnings%3A%3Aearnings-call-transcripts&filter[since]=0"\
      +"&filter[until]=0&include=author%2CprimaryTickers%2CsecondaryTickers&isMounting=true&page[size]="\
      +str(nbr_earning_call_metada)\
      +"&page[number]=1"
url

'https://seekingalpha.com/api/v3/articles?cacheBuster=2022-04-28&filter[category]=earnings%3A%3Aearnings-call-transcripts&filter[since]=0&filter[until]=0&include=author%2CprimaryTickers%2CsecondaryTickers&isMounting=true&page[size]=50&page[number]=1'

In [7]:
# the chrome driver depend on which version of chrome you are running specify the good one
executable_path = "./chromedriver"
service = Service(executable_path)
option = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service,options=option)
driver.get(url)
api_json_response = driver.find_element(by=By.TAG_NAME, value = "body").text
#a quick loock of the result to check if we get results are get captcha/bot error 

print(api_json_response[0:100])
api_data_response = json.loads(api_json_response)

{"data":[{"id":"4507842","type":"transcript","attributes":{"publishOn":"2022-05-06T08:05:04-04:00","


## API data response

<pre>
dict( 
    data : list(one element is one earning call metada)
    included : "don't care for now"
    meta : "don't care for now"
        )
</pre>
One earning call metadata is show below 

In [8]:
api_data_response["data"][0]

{'id': '4507842',
 'type': 'transcript',
 'attributes': {'publishOn': '2022-05-06T08:05:04-04:00',
  'isLockedPro': False,
  'commentCount': 0,
  'gettyImageUrl': None,
  'themes': {},
  'title': 'ING Groep NV (ING) CEO Steven van Rijswijk on Q1 2022 Results - Earnings Call Transcript'},
 'relationships': {'author': {'data': {'id': '44211', 'type': 'author'}},
  'sentiments': {'data': []},
  'primaryTickers': {'data': [{'id': '3211', 'type': 'tag'},
    {'id': '16015', 'type': 'tag'}]},
  'secondaryTickers': {'data': []},
  'otherTags': {'data': [{'id': '96991', 'type': 'tag'},
    {'id': '17862', 'type': 'tag'},
    {'id': '49', 'type': 'tag'},
    {'id': '586376', 'type': 'tag'},
    {'id': '18082', 'type': 'tag'},
    {'id': '17875', 'type': 'tag'}]}},
 'links': {'self': '/article/4507842-ing-groep-nv-ing-ceo-steven-van-rijswijk-on-q1-2022-results-earnings-call-transcript'}}

In [9]:

id_and_path_to_earning_calls = [ (earning_call_metada["id"] ,earning_call_metada["links"]["self"]) 
                     for earning_call_metada in api_data_response["data"]]
# ""check url
id_and_path_to_earning_calls[0:2]

[('4507842',
  '/article/4507842-ing-groep-nv-ing-ceo-steven-van-rijswijk-on-q1-2022-results-earnings-call-transcript'),
 ('4507754',
  '/article/4507754-confluent-inc-cflt-ceo-jay-kreps-on-q1-2022-results-earnings-call-transcript')]

## download all not yet download html from url

In [ ]:
domain_name = 'https://seekingalpha.com'
need_to_download = []
for id_earning_call, path_to_earning_call in tqdm(id_and_path_to_earning_calls):
    if not os.path.isfile("data/article/"+ id_earning_call +".html"):
        # not in local directory
        need_to_download.append((id_earning_call, path_to_earning_call))
print(f"Need to download {len(need_to_download)} html file(s)")
for id_earning_call, path_to_earning_call in tqdm(need_to_download):
    with open("data/article/"+ id_earning_call+".html", "x") as file:
            url = domain_name + path_to_earning_call
            html_transcript = requests.get(url).text
            file.write(html_transcript)
    

100%|████████████████████████████████████████| 50/50 [00:00<00:00, 23234.57it/s]


Need to download 50 html file(s)


 22%|█████████▍                                 | 11/50 [00:05<00:19,  2.04it/s]

## Check last file got downloaded, i.e. captcha/bot error 

In [ ]:
BeautifulSoup(html_transcript).get_text()[:2000]